In [1]:
import numpy as np
import pandas as pd
import os
import pypsa
import plotly
import plotly.express as px
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import plotly.graph_objects as go

%matplotlib inline

language = "PL"

In [2]:
colors = ['black', '#c0843f', 'rgb(95, 70, 144)', 'rgb(29, 105, 150)', 'rgb(56, 166, 165)', 'rgb(15, 133, 84)',
          'rgb(115, 175, 72)', 'rgb(237, 173, 8)', 'rgb(225, 124, 5)', 'rgb(204, 80, 62)', 'rgb(148, 52, 110)',
          'rgb(111, 64, 112)', 'rgb(102, 102, 102)']

def fuel_colors(fuel):
    fuel_colors_dict = {
        'hard coal': 'black',
        'lignite': '#c0843f',
        'Fossil gas': 'rgb(170, 170, 170)',
        'Oil': 'rgb(102, 102, 102)',
        'Wind onshore': 'rgb(204, 80, 62)',
        'Wind offshore': 'rgb(145, 44, 29)',
        'PV': 'rgb(237, 173, 8)',
        'Hydro': 'rgb(29, 105, 150)',
        'Hydro Pumped Storage': 'rgb(56, 166, 165)',
        'Hydro Run-of-river and poundage': 'rgb(29, 105, 150)',
        'Hydro Water Reservoir': 'rgb(29, 105, 150)',
        'biogas': 'rgb(10, 89, 56)',
        'biomass': 'rgb(15, 133, 84)',
        'geothermal': 'rgb(139, 70, 144)',
        'Other fuels': 'rgb(95, 70, 144)',
        'Import': 'rgb(95, 70, 144)',
        'ZEPAK': 'rgb(135, 100, 59)',
        'Battery': 'rgb(237, 138, 8)',
    }
    return fuel_colors_dict[fuel]

def fuel_name(fuel):
    fuel_name_dict = {
        'hard coal': 'Węgiel kamienny' if language == "PL" else 'Hard coal',
        'lignite': 'Węgiel brunanty' if language == "PL" else 'Lignite',
        'Fossil gas': 'Gaz ziemny' if language == "PL" else 'Natural gas',
        'Oil': 'Ropa naftowa' if language == "PL" else 'Oil',
        'Wind onshore': 'Wiatr na lądzie' if language == "PL" else 'Wind - onshore',
        'Wind offshore': 'Wiatr na morzu' if language == "PL" else 'Wind - offshore',
        'PV': 'Słońce' if language == "PL" else 'PV',
        'Hydro': 'Woda - przepływowe' if language == "PL" else 'Hydro',
        'Hydro Pumped Storage': 'Woda - ESP' if language == "PL" else 'Pumped-storage',
        'biogas': 'Biogaz' if language == "PL" else 'Biogas',
        'biomass': 'Biomasa' if language == "PL" else 'Biomass',
        'geothermal': 'Energia geotermalna' if language == "PL" else 'Geothermal energy',
        'Other fuels': 'Inne' if language == "PL" else 'Other',
        'Import': 'Import' if language == "PL" else 'Import',
        'ZEPAK': 'ZEPAK',
        'Battery': 'Magazyny Li-Ion' if language == "PL" else 'Energy storage Li-Ion',
    }
    return fuel_name_dict[fuel]


# Get load and wind & pv efficiency

In [3]:
load = pd.read_excel('RESPotentialInEntreprisesData.xlsx', sheet_name='load', index_col=0, parse_dates=True)
load

,warehouse,supermarket,strip mall,shopping center
t,,,,
2019-01-01 00:00:00,0.112553,0.470144,0.556930,1.027074
2019-01-01 01:00:00,0.602876,0.503453,0.557817,1.061269
2019-01-01 02:00:00,0.583166,0.504479,0.559578,1.064058
2019-01-01 03:00:00,0.497459,0.483369,0.561139,1.044508
2019-01-01 04:00:00,0.581825,0.642756,0.562669,1.205425
...,...,...,...,...
2019-12-31 19:00:00,0.588127,1.203122,1.783591,2.986713
2019-12-31 20:00:00,0.573899,1.063551,1.616155,2.679705
2019-12-31 21:00:00,0.494693,0.751376,0.823932,1.575308


In [4]:
load.sum()

warehouse           7977.513748
supermarket         8908.000000
strip mall         10047.000000
shopping center    18955.000000
dtype: float64

In [5]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=load.index, y=load["warehouse"], name='Load [MW]' if language == "EN" else 'Zapotrzebowanie [MW]', line=dict(color='rgb(29, 105, 150)', width=2))) #, fill='tozeroy', line=dict(color='black', width=2)))

fig.update_layout(
        width=1000,
        plot_bgcolor='white',  
        title=dict(
            text='<b>Pumped storage usage</b>' if language == "EN" else '<b>Profil użycia elektrowni szczytowo-pompowych</b>',
            font=dict(
                family="Work Sans, sans-serif",
                size=13,
                color="black",
            ),
            x=0,
            xref='paper',
            yref='paper',
            xanchor="left", 
            yanchor="top",
        ),
        margin=dict(t=80, b=80,l=50, r=50),
        font=dict(
            family="Work Sans, sans-serif",
            size=11,
            color="grey"
        ),
        annotations=[
            go.layout.Annotation( 
                text='<i>Pumped storage units are charged during the night and dispatched during peak day hours.</i>' if language == "EN" else '<i>Elektrownie szczytowo-pompowe gromadzą energię w nocy i oddają ją w dziennym szczycie zapotrzebowania.</i>',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0,
                y=1.1,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
            go.layout.Annotation( 
                text='<i>Sources: Instrat, CC BY 4.0.</i>' if language == "EN" else '<i>Źródło: Instrat, CC BY 4.0.</i>',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0,
                y=-0.19,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
        ]
)

fig.update_yaxes(
        title=dict(
            text='Pumped storage charge [MWh] and dispatch [MW]' if language == "EN" else 'Stan naładowania [MWh] i generacja/pompowanie [MW]',
            font=dict(
                family="Work Sans, sans-serif",
                size=11,
                color="grey",
            )
        ),
    showgrid=True,
    gridwidth=1, 
    gridcolor='LightGrey',
    zeroline=True, 
    zerolinewidth=2, 
    zerolinecolor='DarkGrey'
)

fig.show(    
    config={
        'modeBarButtonsToRemove': ['lasso2d', "zoomIn2d", 'resetScale2d'],
        "locale": "en",
        "toImageButtonOptions": {"width": None, "height": None}
    },
)

In [6]:
pv = pd.read_excel('RESPotentialInEntreprisesData.xlsx', sheet_name='pv', index_col=0, parse_dates=True)
pv

,wielkopolskie,mazowieckie,sroda,warszawa
t,,,,
2019-01-01 00:00:00,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,0.0,0.0,0.0,0.0
...,...,...,...,...
2019-12-31 19:00:00,0.0,0.0,0.0,0.0
2019-12-31 20:00:00,0.0,0.0,0.0,0.0
2019-12-31 21:00:00,0.0,0.0,0.0,0.0


In [7]:
pv.mean()

wielkopolskie    0.134100
mazowieckie      0.132025
sroda            0.134176
warszawa         0.132454
dtype: float64

In [8]:
wind = pd.read_excel('RESPotentialInEntreprisesData.xlsx', sheet_name='wind', index_col=0, parse_dates=True)
wind

,wielkopolskie,mazowieckie,sroda_80m,sroda_20m,warszawa_80m,warszawa_20m
t,,,,,,
2019-01-01 00:00:00,0.383616,0.266371,0.358,0.171,0.246,0.115
2019-01-01 01:00:00,0.383616,0.266371,0.358,0.171,0.246,0.115
2019-01-01 02:00:00,0.437379,0.338995,0.414,0.202,0.317,0.154
2019-01-01 03:00:00,0.503910,0.414993,0.479,0.238,0.386,0.196
2019-01-01 04:00:00,0.564968,0.448373,0.547,0.281,0.434,0.229
...,...,...,...,...,...,...
2019-12-31 19:00:00,0.556477,0.678545,0.480,0.243,0.593,0.308
2019-12-31 20:00:00,0.492017,0.592206,0.415,0.214,0.491,0.251
2019-12-31 21:00:00,0.440431,0.618472,0.353,0.179,0.484,0.253


In [9]:
wind.mean()

wielkopolskie    0.231458
mazowieckie      0.235084
sroda_80m        0.263367
sroda_20m        0.122915
warszawa_80m     0.261249
warszawa_20m     0.121764
dtype: float64

# Model assumptions #1

### Emissions (tCO2 / 1 MWh)

In [10]:
# Data from KOBiZE

coal_co2_emissions = 0.33
lignite_co2_emissions = 0.38
gas_co2_emissions = 0.19
biomass_co2_emissions = 0.403
biogas_co2_emissions = 0.178

water_co2_emissions = 0.
solar_co2_emissions = 0.
onwind_co2_emissions = 0.

offwind_co2_emissions = 0.
nuclear_co2_emissions = 0.

### Currencies

In [11]:
eur_to_pln = 4.42
usd_to_pln = 3.76
pound_to_pln = 4.90

### CO2 Price (EUR / tCO2)

In [12]:
co2_price = 25 * eur_to_pln # energy.instrat.pl

### Life expectancy (years)

In [13]:
# source - fisrt pages of google

coal_life_expectancy = 40
lignite_life_expectancy = 40
gas_life_expectancy = 30
biomass_life_expectancy = 30

water_life_expectancy = 50
solar_life_expectancy = 30
onwind_life_expectancy = 25

offwind_life_expectancy = 20
nuclear_life_expectancy = 60

### Annual operating costs (PLN / MW / a)

In [14]:
# Data from calliope excel. Those prices vary significantly depending on source
# Other source: https://usea.org/sites/default/files/Operating%20ratio%20and%20cost%20of%20coal%20power%20generation%20-%20ccc272-1.pdf
# Other source: https://en.wikipedia.org/wiki/Cost_of_electricity_by_source
# Other source: https://www.eia.gov/todayinenergy/detail.php?id=31912

biomass_annual_operating_cost = 50 * eur_to_pln * 1000
solar_annual_operating_cost = 15 * usd_to_pln * 1000
onwind_annual_operating_cost = 30 * usd_to_pln * 1000

### Cost of building 1 MW

In [15]:
# Source problem same as above

biogas_1MW_investment_cost = 13260000
biomass_1MW_investment_cost = 15403000
solar_1MW_investment_cost = 2300000
onwind_1MW_investment_cost = 3900000
battery_1MW_investment_cost = 840000*4

### Investment cost

### Fuel cost (PLN / MWh primary)

In [16]:
# price [PLN or USD per tonne] / calorific value [GJ per tonne] / 0.28 [GJ to MWh]

coal_fuel_cost = 12.11 / 0.28 # energy.instrat.pl
lignite_fuel_cost = 22 # https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwiA39_cnPLqAhWSlosKHVzuAssQFjACegQIAxAB&url=http%3A%2F%2Fyadda.icm.edu.pl%2Fyadda%2Felement%2Fbwmeta1.element.baztech-article-BPB9-0003-0002%2Fc%2Fhttpwww_min-pan_krakow_plwydawnictwape122211-czopek-trzaskus-zak.pdf&usg=AOvVaw2KxhSn2VsYSY7-WWUAnRAq
gas_fuel_cost = 14 * eur_to_pln # https://ec.europa.eu/energy/sites/ener/files/quarterly_report_on_european_gas_markets_q4_2019_final.pdf
biomass_fuel_cost = 83 * usd_to_pln / 15 / 0.28 # http://greenwoodresources.com/wp-content/uploads/2018/04/A-Biomass-Production-Cost-Calculator-PDF.pdf, http://www.instalacjebudowlane.pl/5173-33-68-biomasa--cena-wartosc-opalowa-rodzaje.html#nowhere

water_fuel_cost = 0
solar_fuel_cost = 0
onwind_fuel_cost = 0

offwind_fuel_cost = 0
nuclear_fuel_cost = 1390 * usd_to_pln / 82000 / 0.28 # https://www.world-nuclear.org/information-library/economic-aspects/economics-of-nuclear-power.aspx, https://www.world-nuclear.org/information-library/nuclear-fuel-cycle/introduction/physics-of-nuclear-energy.aspx

In [17]:
biomass_fuel_cost, coal_fuel_cost, nuclear_fuel_cost, lignite_fuel_cost

(74.3047619047619, 43.24999999999999, 0.22763066202090587, 22)

### Operating costs [PLN / MWh]

In [18]:
# price [PLN or USD per tonne] / calorific value [GJ per tonne] / 0.28 [GJ to MWh]

biomass_operating_cost = 5 * usd_to_pln
solar_operating_cost = 0
onwind_operating_cost = 0

### OPEX

In [19]:
def mwh_production_cost(co2_price, co2_emissions, fuel_cost, power_plant_efficiency, operating_cost):
    return co2_price * co2_emissions + fuel_cost / power_plant_efficiency + operating_cost

### Ramp limit [MW / min]

In [20]:
coal_ramp_limit = 25 # https://www.senat.gov.pl/gfx/senat/userfiles/_public/k9/komisje/2019/kgni/materialy/bujalski_technologie.pdf
gas_ramp_limit = 85 #https://www.senat.gov.pl/gfx/senat/userfiles/_public/k9/komisje/2019/kgni/materialy/bujalski_technologie.pdf

# Model

## OPEX only

In [370]:
n = pypsa.Network()
n.set_snapshots(load.index)
n.add('Bus', 'facility')

In [371]:
n.add('Load', 'load', bus='facility', p_set=load['warehouse'])

In [372]:
# tonnes of CO2 eqv. per MWh for each carrier

n.add('Carrier', 'biomass', co2_emissions=biomass_co2_emissions)
n.add('Carrier', 'solar', co2_emissions=0)
n.add('Carrier', 'wind', co2_emissions=0)
n.add('Carrier', 'battery', co2_emissions=0)

In [373]:
n.add('Generator', 'Solar', bus='facility', p_nom=1, carrier='solar', marginal_cost=0, p_max_pu=pv['wielkopolskie'])
n.add('Generator', 'Wind', bus='facility', p_nom=2, carrier='wind', marginal_cost=0, p_max_pu=wind['wielkopolskie'])
n.add('Generator', 'Biomass', bus='facility', p_nom=1, carrier='biomass', marginal_cost=50, efficiency=0.35)
n.add('StorageUnit', 'Battery', bus='facility', p_nom=2.5, carrier='battery', marginal_cost=0, efficiency_dispatch=0.92, state_of_charge_initial=0, cyclic_state_of_charge=True, max_hours=4, efficiency_store=1, standing_loss=0.01, )

In [374]:
# tonnes of CO2 eqv. emissions constraint
# in 2018 Poland emitted 303 mln tonnes of CO2 (https://appsso.eurostat.ec.europa.eu/nui/show.do?dataset=env_ac_ainah_r2&lang=en)

# n.add('GlobalConstraint', 'CO2 limit', sense='<=', constant=303 * 10e6) 

In [375]:
# solve the network by the lowest energy production cost and co2 constraint 

n.lopf(snapshots=n.snapshots, solver_name='cplex', pyomo=False)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.76s
INFO:pypsa.linopf:Solve linear problem using Cplex solver
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 105120 rows and 8761 columns.
Reduced LP has 17520 rows, 43800 columns, and 70080 nonzeros.
Presolve time = 0.13 sec. (67.28 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             0.000000
Iteration:   540   Dual objective     =         34315.372282
Iteration:  1125   Dual objective     =         64765.670661
Iteration:  1667   Dual objective     =         90679.195484
Iteration:  2292   Dual objective     =        101154.755315
Ite

('ok', 'optimal')

In [376]:
p_by_carrier = n.generators_t.p.groupby(n.generators.carrier, axis=1).sum()
p_by_carrier.sum()

carrier
biomass    3320.643531
solar      1146.626571
wind       3673.898523
dtype: float64

In [377]:
# Make subplots like in my excercise - 25 january winter peak, 26 june summer peak
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing = 0.06, vertical_spacing = 0.01)

# p_by_carrier_ZEPAK_summer = p_by_carrier_ZEPAK_ref_2030.loc['2030-06-24 00:00:00':'2030-06-30 00:00:00']
# p_by_carrier_no_ZEPAK_summer = p_by_carrier_no_ZEPAK_ref_2030.loc['2030-06-24 00:00:00':'2030-06-30 00:00:00']
# links_sum_summer = links_sum_ref_2030.loc['2030-06-24 00:00:00':'2030-06-30 00:00:00']
# links_sum_summer = links_sum_summer[(links_sum_summer >= 0)]

# p_by_carrier_ZEPAK_winter = p_by_carrier_ZEPAK_ref_2030.loc['2030-01-21 00:00:00':'2030-01-27 00:00:00']
# p_by_carrier_no_ZEPAK_winter = p_by_carrier_no_ZEPAK_ref_2030.loc['2030-01-21 00:00:00':'2030-01-27 00:00:00']
# links_sum_winter = links_sum_ref_2030.loc['2030-01-21 00:00:00':'2030-01-27 00:00:00']
# links_sum_winter = links_sum_winter[(links_sum_winter >= 0)]

fig.add_trace(go.Scatter(x=p_by_carrier.index, y=p_by_carrier['solar'], stackgroup='one', mode='none', name=fuel_name('PV'), fillcolor='rgba(237, 173, 8, 0.6)', legendgroup='group1'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_by_carrier.index, y=p_by_carrier['wind'], stackgroup='one', mode='none', name=fuel_name('Wind onshore'), fillcolor='rgba(204, 80, 62, 0.6)', legendgroup='group2'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_by_carrier.index, y=p_by_carrier['biomass'], stackgroup='one', mode='none', name=fuel_name('biomass'), fillcolor='rgba(15, 133, 84, 0.6)', legendgroup='group3'), row=1, col=1)

# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['hard coal'], stackgroup='one', mode='none', name=fuel_name('hard coal'), fillcolor='rgba(0,0,0,0.6)', legendgroup='group1', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['lignite'], stackgroup='one', mode='none', name=fuel_name('lignite'), fillcolor='rgba(192, 132, 63,0.6)', legendgroup='group2', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['Fossil gas'], stackgroup='one', mode='none', name=fuel_name('Fossil gas'), fillcolor='rgba(170, 170, 170,0.6)', legendgroup='group3', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['Wind'], stackgroup='one', mode='none', name=fuel_name('Wind onshore'), fillcolor='rgba(204, 80, 62, 0.6)', legendgroup='group4', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['Wind_Off'], stackgroup='one', mode='none', name=fuel_name('Wind offshore'), fillcolor='rgba(145, 44, 29, 0.6)', legendgroup='group5', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['PV'], stackgroup='one', mode='none',name=fuel_name('PV'), fillcolor='rgba(237, 173, 8, 0.6)', legendgroup='group6', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['biomass'], stackgroup='one', mode='none',name=fuel_name('biomass'), fillcolor='rgba(15, 133, 84, 0.6)', legendgroup='group7', showlegend = False), row=1, col=2) 
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['biogas'], stackgroup='one', mode='none',name=fuel_name('biogas'), fillcolor='rgba(10, 89, 56, 0.6)', legendgroup='group8', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['ror'], stackgroup='one', mode='none',name=fuel_name('Hydro'), fillcolor='rgba(29, 105, 150, 0.6)', legendgroup='group9', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=links_sum_summer.index, y=links_sum_summer, stackgroup='one', mode='none', name=fuel_name('Import'), fillcolor='rgba(95, 70, 144, 0.6)', legendgroup='group10', showlegend = False), row=1, col=2)

fig.update_layout(
        width=1000,
        plot_bgcolor='white',  
        title=dict(
            text='<b>Electricity generation by technology [MWh] - BASE scenario - 2030</b>' if language == "EN" else '<b>Produkcja energii elektrycznej per technologia [MWh] - scenariusz bazowy - 2030</b>',
            font=dict(
                family="Work Sans, sans-serif",
                size=13,
                color="black",
            ),
            x=0,
            xref='paper',
            yref='paper',
            xanchor="left", 
            yanchor="top",
        ),
        margin=dict(t=80, b=80,l=50, r=50),
        font=dict(
            family="Work Sans, sans-serif",
            size=11,
            color="grey"
        ),
        annotations=[
            go.layout.Annotation( 
                text='Winter',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0,
                y=1,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
            go.layout.Annotation( 
                text='Summer',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0.53,
                y=1,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
            go.layout.Annotation( 
                text='<i>In 2030, good weather conditions enable RES shares above 50%.</i>' if language == "EN" else '<i>W roku 2030, przy sprzyjających warunkach OZE generują powyżej 50% energii elektrycznej.</i>' ,
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0,
                y=1.1,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
            go.layout.Annotation( 
                text='<i>Sources: Instrat, CC BY 4.0.</i>' if language == "EN" else '<i>Źródło: Instrat, CC BY 4.0.</i>',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0,
                y=-0.19,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
        ]
)

fig.update_yaxes(
        title=dict(
            text='Electricity generation [MWh]' if language == "EN" else 'Produkcja energii elektrycznej [MWh]',
            font=dict(
                family="Work Sans, sans-serif",
                size=11,
                color="grey",
            )
        ),
    showgrid=True,
    gridwidth=1, 
    gridcolor='LightGrey'
)

fig.show(    
    config={
        'modeBarButtonsToRemove': ['lasso2d', "zoomIn2d", 'resetScale2d'],
        "locale": "en",
        "toImageButtonOptions": {"width": None, "height": None}
    },
)

## CAPEX optimization

In [50]:
n = pypsa.Network()
n.set_snapshots(load.index)

n.add('Bus', 'facility')

n.add('Load', 'load', bus='facility', p_set=load['warehouse'])

n.add('Carrier', 'biomass', co2_emissions=biomass_co2_emissions)
n.add('Carrier', 'biogas', co2_emissions=biogas_co2_emissions)
n.add('Carrier', 'solar', co2_emissions=0)
n.add('Carrier', 'wind', co2_emissions=0)
n.add('Carrier', 'battery', co2_emissions=0)

In [51]:
# Let model choose the best energy mix if none is installed

n.add('Generator', 
      'Biomass', 
      bus='facility', 
      p_nom_extendable=False, 
      p_nom=0,
      capital_cost=biomass_1MW_investment_cost/biomass_life_expectancy,
      carrier='biomass',
      marginal_cost=mwh_production_cost(co2_price, biomass_co2_emissions, biomass_fuel_cost, 0.35, biomass_operating_cost), 
      efficiency=0.35)

n.add('Generator', 
      'Biogas', 
      bus='facility', 
      p_nom_extendable=True, 
      #p_nom_max=1,
      capital_cost=biogas_1MW_investment_cost/biomass_life_expectancy,
      carrier='biogas',
      marginal_cost=mwh_production_cost(co2_price, biomass_co2_emissions, biomass_fuel_cost, 0.35, biomass_operating_cost), 
      efficiency=0.35)

n.add('Generator', 
      'Solar', 
      bus='facility', 
      p_nom_extendable=True, 
      #p_nom_max=2,
      capital_cost=solar_1MW_investment_cost/solar_life_expectancy,
      carrier='solar', 
      marginal_cost=0, 
      p_max_pu=pv['wielkopolskie'])

n.add('Generator', 
      'Wind', 
      bus='facility', 
      p_nom_extendable=True, 
      #p_nom_max=3.3, 
      capital_cost=onwind_1MW_investment_cost/onwind_life_expectancy,
      carrier='wind', 
      marginal_cost=0, 
      p_max_pu=wind['sroda_80m'])

n.add('StorageUnit', 'Battery', bus='facility', p_nom_extendable=True, carrier='battery', marginal_cost=0, capital_cost=battery_1MW_investment_cost/20, efficiency_dispatch=0.92, state_of_charge_initial=0, cyclic_state_of_charge=True, max_hours=4, efficiency_store=1, standing_loss=0.01)

In [52]:
n.lopf(snapshots=n.snapshots, solver_name='cplex', pyomo=False)

INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.78s
INFO:pypsa.linopf:Solve linear problem using Cplex solver
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Parallel mode: deterministic, using up to 12 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 10 threads...
 * Starting primal Simplex on 1 thread...
Tried aggregator 1 time.
LP Presolve eliminated 74467 rows and 13148 columns.
Reduced LP has 65693 rows, 48177 columns, and 170799 nonzeros.
Presolve time = 0.14 sec. (91.65 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =             0.000000
Perturbation started.
Iteration:   101   Dual objective     =             0.000000
Iteration:   274   Dual objective     =        585812.025333
Iteration:   543   Dual objective     =        608692.339112
Iteration:   808   Dual objective     =  

('ok', 'optimal')

In [53]:
n.generators.p_nom_opt

Biomass    0.000000
Biogas     1.816733
Solar      8.256137
Wind       5.795847
Name: p_nom_opt, dtype: float64

In [36]:
generator_cost=(n.generators.p_nom_opt*n.generators.capital_cost*30)/1000000
storage_cost = (n.storage_units.p_nom_opt*n.storage_units.capital_cost)/1000000
generator_cost.sum() + storage_cost.sum()

28.17965943196163

In [31]:
n.storage_units.p_nom_opt

Battery    1.755444
Name: p_nom_opt, dtype: float64

In [32]:
p_by_carrier = n.generators_t.p.groupby(n.generators.carrier, axis=1).sum()
p_by_carrier.sum()

carrier
biogas      874.603842
biomass       0.000000
solar      4395.665247
wind       3127.230469
dtype: float64

In [37]:
# Make subplots like in my excercise - 25 january winter peak, 26 june summer peak
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing = 0.06, vertical_spacing = 0.01)

# p_by_carrier_ZEPAK_summer = p_by_carrier_ZEPAK_ref_2030.loc['2030-06-24 00:00:00':'2030-06-30 00:00:00']
# p_by_carrier_no_ZEPAK_summer = p_by_carrier_no_ZEPAK_ref_2030.loc['2030-06-24 00:00:00':'2030-06-30 00:00:00']
# links_sum_summer = links_sum_ref_2030.loc['2030-06-24 00:00:00':'2030-06-30 00:00:00']
# links_sum_summer = links_sum_summer[(links_sum_summer >= 0)]

# p_by_carrier_ZEPAK_winter = p_by_carrier_ZEPAK_ref_2030.loc['2030-01-21 00:00:00':'2030-01-27 00:00:00']
# p_by_carrier_no_ZEPAK_winter = p_by_carrier_no_ZEPAK_ref_2030.loc['2030-01-21 00:00:00':'2030-01-27 00:00:00']
# links_sum_winter = links_sum_ref_2030.loc['2030-01-21 00:00:00':'2030-01-27 00:00:00']
# links_sum_winter = links_sum_winter[(links_sum_winter >= 0)]

fig.add_trace(go.Scatter(x=p_by_carrier.index, y=p_by_carrier['solar'], stackgroup='one', mode='none', name=fuel_name('PV'), fillcolor='rgba(237, 173, 8, 0.6)', legendgroup='group1'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_by_carrier.index, y=p_by_carrier['wind'], stackgroup='one', mode='none', name=fuel_name('Wind onshore'), fillcolor='rgba(204, 80, 62, 0.6)', legendgroup='group2'), row=1, col=1)
fig.add_trace(go.Scatter(x=p_by_carrier.index, y=p_by_carrier['biogas'], stackgroup='one', mode='none', name=fuel_name('biogas'), fillcolor='rgba(15, 133, 84, 0.6)', legendgroup='group3'), row=1, col=1)

# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['hard coal'], stackgroup='one', mode='none', name=fuel_name('hard coal'), fillcolor='rgba(0,0,0,0.6)', legendgroup='group1', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['lignite'], stackgroup='one', mode='none', name=fuel_name('lignite'), fillcolor='rgba(192, 132, 63,0.6)', legendgroup='group2', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['Fossil gas'], stackgroup='one', mode='none', name=fuel_name('Fossil gas'), fillcolor='rgba(170, 170, 170,0.6)', legendgroup='group3', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['Wind'], stackgroup='one', mode='none', name=fuel_name('Wind onshore'), fillcolor='rgba(204, 80, 62, 0.6)', legendgroup='group4', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['Wind_Off'], stackgroup='one', mode='none', name=fuel_name('Wind offshore'), fillcolor='rgba(145, 44, 29, 0.6)', legendgroup='group5', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['PV'], stackgroup='one', mode='none',name=fuel_name('PV'), fillcolor='rgba(237, 173, 8, 0.6)', legendgroup='group6', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['biomass'], stackgroup='one', mode='none',name=fuel_name('biomass'), fillcolor='rgba(15, 133, 84, 0.6)', legendgroup='group7', showlegend = False), row=1, col=2) 
#fig.add_trace(go.Scatter(x=p_by_carrier.index, y=p_by_carrier['biogas'], stackgroup='one', mode='none',name=fuel_name('biogas'), fillcolor='rgba(10, 89, 56, 0.6)', legendgroup='group8', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=p_by_carrier_no_ZEPAK_summer.index, y=p_by_carrier_no_ZEPAK_summer['ror'], stackgroup='one', mode='none',name=fuel_name('Hydro'), fillcolor='rgba(29, 105, 150, 0.6)', legendgroup='group9', showlegend = False), row=1, col=2)
# fig.add_trace(go.Scatter(x=links_sum_summer.index, y=links_sum_summer, stackgroup='one', mode='none', name=fuel_name('Import'), fillcolor='rgba(95, 70, 144, 0.6)', legendgroup='group10', showlegend = False), row=1, col=2)

fig.update_layout(
        width=1000,
        plot_bgcolor='white',  
        title=dict(
            text='<b>Electricity generation by technology [MWh] - BASE scenario - 2030</b>' if language == "EN" else '<b>Produkcja energii elektrycznej per technologia [MWh] - scenariusz bazowy - 2030</b>',
            font=dict(
                family="Work Sans, sans-serif",
                size=13,
                color="black",
            ),
            x=0,
            xref='paper',
            yref='paper',
            xanchor="left", 
            yanchor="top",
        ),
        margin=dict(t=80, b=80,l=50, r=50),
        font=dict(
            family="Work Sans, sans-serif",
            size=11,
            color="grey"
        ),
        annotations=[
            go.layout.Annotation( 
                text='Winter',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0,
                y=1,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
            go.layout.Annotation( 
                text='Summer',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0.53,
                y=1,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
            go.layout.Annotation( 
                text='<i>In 2030, good weather conditions enable RES shares above 50%.</i>' if language == "EN" else '<i>W roku 2030, przy sprzyjających warunkach OZE generują powyżej 50% energii elektrycznej.</i>' ,
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0,
                y=1.1,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
            go.layout.Annotation( 
                text='<i>Sources: Instrat, CC BY 4.0.</i>' if language == "EN" else '<i>Źródło: Instrat, CC BY 4.0.</i>',
                align='left',
                showarrow=False,
                xref='paper',
                yref='paper',
                x=0,
                y=-0.19,
                xanchor="left", 
                yanchor="top",
                visible=True,
                font=dict(
                    family="Work Sans, sans-serif",
                    size=11,
                    color="grey"
                )
            ),
        ]
)

fig.update_yaxes(
        title=dict(
            text='Electricity generation [MWh]' if language == "EN" else 'Produkcja energii elektrycznej [MWh]',
            font=dict(
                family="Work Sans, sans-serif",
                size=11,
                color="grey",
            )
        ),
    showgrid=True,
    gridwidth=1, 
    gridcolor='LightGrey'
)

fig.show(    
    config={
        'modeBarButtonsToRemove': ['lasso2d', "zoomIn2d", 'resetScale2d'],
        "locale": "en",
        "toImageButtonOptions": {"width": None, "height": None}
    },
)